In [5]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
import csv, os
import pandas as pd
import xlsxwriter

# download all reference of proteome for each species  in the format of fasta to a folder

In [10]:
os.chdir(os.curdir)

In [18]:
ls 

Arabidopsis.fasta*                    ecoli.fasta_slected.xlsx*
Arabidopsis.fasta_slected.xlsx*       gorila.fasta*
Archive 2/                            gorila.fasta_slected.xlsx*
Archive 2.zip*                        human.fasta*
FLAT_FILES_072010/                    human.fasta_slected.xlsx*
Macaca.fasta*                         mouse.fasta*
Macaca.fasta_slected.xlsx*            mouse.fasta_slected.xlsx*
Protein Analysis-common-models.ipynb* rice.fasta*
Rattus.fasta*                         rice.fasta_slected.xlsx*
Rattus.fasta_slected.xlsx*            xenopus.fasta*
celegans.fasta*                       xenopus.fasta_slected.xlsx*
celegans.fasta_slected.xlsx*          yeast.fasta*
drosophila.fasta*                     yeast.fasta_slected.xlsx*
drosophila.fasta_slected.xlsx*        zebrafish.fasta*
ecoli.fasta*                          zebrafish.fasta_slected.xlsx*


In [13]:
def list_files(file_path='./', file_ext = 'pdf'):
    ### list chosen file types in a folder, output into a list
    from os import listdir
    from os.path import isfile, join
    all_files = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    pdf_files = [x for x in all_files if x[-len(file_ext):] == file_ext ] 
#     for pdf in pdf_files:
#         print (pdf)
    return pdf_files

In [19]:
protein_refs=list_files(file_ext='fasta')

In [20]:
protein_refs

['Macaca.fasta',
 'celegans.fasta',
 'yeast.fasta',
 'drosophila.fasta',
 'Arabidopsis.fasta',
 'mouse.fasta',
 'ecoli.fasta',
 'human.fasta',
 'Rattus.fasta',
 'zebrafish.fasta',
 'xenopus.fasta',
 'gorila.fasta',
 'rice.fasta']

In [16]:
def select_poly_de_proteins(ref_fasta):
    record_dict = SeqIO.to_dict(SeqIO.parse(ref_fasta, "fasta"))
    selected = dict()
    for rid, record in record_dict.items():
    #     print (rid, record)
        my_seq=str(record.seq.upper())
        tmp=dict()
        tmp['full_seq']=my_seq
    #     tmp["rid"]=rid
    #     print (type(my_seq))
        i=0
        threthold = 40
        window_size=70
        hit=list()
        while i+window_size < len(my_seq):
            curr_seq=my_seq[i:i+window_size]
            ttmp=dict()
            analysed_seq = ProteinAnalysis(curr_seq)
            count_dict= analysed_seq.count_amino_acids()
            if count_dict['D'] + count_dict["E"] >=threthold:
                tmp['D']=count_dict['D']
                tmp['E']=count_dict['E']
                tmp['D_E']=count_dict['D']+count_dict['E']
                tmp['start']=i
                tmp['end']=i+window_size
                
                ttmp['D']=count_dict['D']
                ttmp['E']=count_dict['E']
                ttmp['D_E']=count_dict['D']+count_dict['E']
                ttmp['start']=i
                ttmp['end']=i+window_size
                hit.append(ttmp)
                tmp['all_hits']=hit    #             print (count_dic['D'], count_dic['E'])
            else:
                pass
            i=i+1
        if len(tmp) >1:
            selected[rid] = tmp
    # print (len(selected))
    selected_df=pd.DataFrame(selected)
    selected_df=selected_df.transpose()
    columns_to_write=selected_df.columns.values
    output_name='{}_slected.xlsx'.format(str(ref_fasta))
    writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
    # Write ddeach dataframe to a different worksheet.
    selected_df.to_excel(writer, sheet_name='D_E40_70')
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    print (str(ref_fasta), len(selected))
    return selected

In [24]:
# ### Take multiple hits into consideration
# def select_poly_de_proteins_multiple_hits(ref_fasta,de=40,window=70):
#     record_dict = SeqIO.to_dict(SeqIO.parse(ref_fasta, "fasta"))
#     selected = dict()
#     for rid, record in record_dict.items():
#     #     print (rid, record)
#         my_seq=str(record.seq.upper())
#         tmp=dict()
#         tmp['full_seq']=my_seq
#     #     tmp["rid"]=rid
#     #     print (type(my_seq))
#         i=0
#         threthold = de
#         window_size=window
#         hit=list()
#         while i+window_size < len(my_seq):
    
#             ttmp=dict()
#             curr_seq=my_seq[i:i+window_size]
#     #         print (len(curr_seq))
#             analysed_seq = ProteinAnalysis(curr_seq)
#             count_dict= analysed_seq.count_amino_acids()
#             if count_dict['D'] + count_dict["E"] >=threthold:
#                 ttmp['D']=count_dict['D']
#                 ttmp['E']=count_dict['E']
#                 ttmp['D_E']=count_dict['D']+count_dict['E']
#                 ttmp['start']=i
#                 ttmp['end']=i+window_size
#                 hit.append(ttmp)
#                 tmp['hit']=hit
                
#     #             print (count_dic['D'], count_dic['E'])
#             else:
#                 pass
#             i=i+1
#         if len(tmp) >1:
#             selected[rid] = tmp
#     # print (len(selected))
#     selected_df=pd.DataFrame(selected)
#     selected_df=selected_df.transpose()
#     columns_to_write=selected_df.columns.values
#     output_name='{}_slected_mutiple_hits.xlsx'.format(str(ref_fasta))
#     writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
#     # Write ddeach dataframe to a different worksheet.
#     selected_df.to_excel(writer, sheet_name='D_E40_70')
#     # Close the Pandas Excel writer and output the Excel file.
#     writer.save()
#     print (ref_fasta, len(selected))
#     return selected

In [32]:
poly_de_40_70 = dict()

for ref in protein_refs:
    poly_de_40_70[str(ref)]=select_poly_de_proteins(ref)

    
    

Macaca.fasta 137
celegans.fasta 57
yeast.fasta 24
drosophila.fasta 92
Arabidopsis.fasta 73
mouse.fasta 125
ecoli.fasta 0
human.fasta 107
Rattus.fasta 116
zebrafish.fasta 117
xenopus.fasta 65
gorila.fasta 136
rice.fasta 42
